# nPE: A Configurable Processing Engine
#### Documentation | Version: 0.4.1 | Updated 2018.7.25
---

## Introduction

The nPE is a highly configurable processing engine that can be topologically configured to support various DNN accelerator architectures. Its functionality is able to support inner products with varying amounts of spatial and temporal parallelism, nonlinear activation functions, scalar addition, scalar multiplication, and max functions. It supports many existing architectures and dataflows, such as Eyeriss, Flexflow, and ShiDianNao. Its generality based on the notion of soft, firm, and hard bypasses.
- Soft bypasses make it possible to bypass or not bypass via control signals and a multiplexer.
- Firm bypasses add a bypass line as an output.
- Hard bypasses hardwire the input and output together.

The details of bypass behavior change depending on the specific module.

It's microarchitecture consists of two multiport Register Files, an n-dimensional inner product unit, an RF-with-ALU combo, and a nonlinear processing unit. All three registers are accessible from an external NoC, and all arithmetic modules support an "Identity" operation.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = """
C:\Users\RyanL\OneDrive\Research\SEAL\processing-engine/source/load-ivy.sc
"""

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.math.pow

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}


import scala.math.pow

## Parallel Register File

The Parallel Register File (pRF) is a register file that is configurable based on number of ports in/out, register size, bypass type, and bitwidth. Bypass options are None, Soft, or Hard. The bypass behavior is such that it bypasses in[k] to out[k], where k is the port index. Without loss of generality, the bypass option will be applied to all ports. Multiple bypass options are enabled by parallel pRFs.

### Single Register File

The Single Register File acts as expected: it reads data in when write enable is high and reads data when read enable is high. It can read and write in parallel. It has two read ports: one for internal PE reading and one for external PE communication. The register size adjusts based on the bitwidth of the addresses.

In [3]:
class RFInputs(dataWidth: Int, addrWidth: Int) extends Bundle {
    
    override def cloneType = (new RFInputs(dataWidth, addrWidth)).asInstanceOf[this.type]
    
    val wEnable  = Input(Bool())
    val rEnable  = Input(Bool())
    val wAddr    = Input(UInt(addrWidth.W))
    val wData    = Input(SInt(dataWidth.W))
    val rAddrInt = Input(UInt(addrWidth.W))
    val rAddrExt = Input(UInt(addrWidth.W))
}

class RFOutputs(dataWidth: Int) extends Bundle {
    
    override def cloneType = (new RFOutputs(dataWidth)).asInstanceOf[this.type]
    
    val rDataInt = Output(SInt(dataWidth.W))
    val rDataExt = Output(SInt(dataWidth.W))
}

class RF (dataWidth: Int, addrWidth: Int) extends Module {
  
    val io = IO(new Bundle {
        val in  = new RFInputs(dataWidth, addrWidth)
        val out = new RFOutputs(dataWidth)
    })
    
    val registers  = RegInit(Vec(Seq.fill(pow(2, addrWidth).toInt) { 0.S(addrWidth.W) }))
    
    when(io.in.wEnable) {
        registers(io.in.wAddr) := io.in.wData
    }
    
    when(io.in.rEnable) {
        io.out.rDataInt := registers(io.in.rAddrInt)
        io.out.rDataExt := registers(io.in.rAddrExt)
    } .otherwise {
        io.out.rDataInt := 0.S
        io.out.rDataExt := 0.S
    }
}

defined class RF

### Putting them Together

The single RFs come together in parallel to form the pRF. The number of ports is the number of single RFs. Thus, there are all of the control signals associated with a single RF, but with a bus width equal to the port count. If the bypass type is Soft, then each RF has an independent bypass control signal.

In [4]:
class PRF(ports: Int, bypass: String, dataWidth: Int, addrWidth: Int) extends Module {
    
    require(List("None", "Soft", "Hard").contains(bypass))
    
    val io = IO(new Bundle {
        val in    = Vec(ports, new RFInputs(dataWidth, addrWidth))
        val out   = Vec(ports, new RFOutputs(dataWidth))
        val bpSel = if (bypass == "Soft") Some(Input(Vec(ports, Bool()))) else None
    })
    
    if (bypass == "None" || bypass == "Soft") {
        
        val rf = Seq.fill(ports){ Module(new RF(dataWidth, addrWidth)) }
        
        rf.zipWithIndex.map{ case (x: RF, i: Int) => {
            
            x.io.in.wEnable  := io.in(i).wEnable
            x.io.in.rEnable  := io.in(i).rEnable
            x.io.in.wAddr    := io.in(i).wAddr
            x.io.in.wData    := io.in(i).wData
            x.io.in.rAddrInt := io.in(i).rAddrInt
            x.io.in.rAddrExt := io.in(i).rAddrExt
            
            when (io.bpSel.getOrElse(Seq.fill(ports){ false.B })(i)) {
                io.out(i).rDataInt := x.io.in.wData
                io.out(i).rDataExt := x.io.in.wData
            } .otherwise {
                io.out(i).rDataInt := x.io.out.rDataInt
                io.out(i).rDataExt := x.io.out.rDataExt
            }
        }}
        
    } else if (bypass == "Hard") {
        for(i <- 0 until ports) {
            io.out(i).rDataInt := io.in(i).wData
            io.out(i).rDataExt := io.in(i).wData
        }
    }
}

defined class pRF

## Inner Product Unit

The Inner Product Unit (IPU) performs an inner product between two vectors of a configurable length. For SIMD support, this should be configured to be wide. For traditional PEs, such as Eyeriss, note that setting the width to 1 is equivalent to a scalar multiplication.

Bypass options are None or Firm. For Firm, there only one pair of weights and activations may be bypassed at a time, such that the IPU may interface with only one ALU. Thus if the bypass type is Firm, then there is a bypass selection signal that selects which input to feed to the output. This may be generalized to interface with multiple TUs, but not at this time.

There are plans to implement both Soft and Hard bypassing. There are also plans to make using FMA modules an option.

The IPU contains two embedded modules: a parallel multiplier and a reduction tree.

### Parallel Multiplier

The parallel multiplier simply takes two vector inputs of width n and outputs the element-wise multiplication. There are plans to make the multiplier type configurable, *e.g.* serial, combinational, pipelined *etc*.

In [5]:
class PMultiplier(numPairs: Int, bitWidth: Int) extends Module {
    
    require(numPairs >= 1, "Must have at least one pair of multiplicands.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
    
    val io = IO(new Bundle {
        val in1 = Input (Vec(numPairs, SInt(bitWidth.W)))
        val in2 = Input (Vec(numPairs, SInt(bitWidth.W)))
        val out = Output(Vec(numPairs, SInt(bitWidth.W)))
    })
    
    io.out := (io.in1 zip io.in2).map { case(a, b) => a * b }
}

defined class pMultiplier

### Additive Reduction Tree

The additive reduction tree outputs the L1 norm of the n-dim input vector using log(n) layers of 2-input adders. In other words, it sums the input.

There are plans to pipeline this (as well as everything else...).

In [6]:
// Recursively creates a balanced syntax tree
def nonassocPairwiseReduce[A](xs: List[A], op: (A, A) => A): A = {
  xs match {
    case Nil => throw new IllegalArgumentException
    case List(singleElem) => singleElem
    case sthElse => {
      val grouped = sthElse.grouped(2).toList
      val pairwiseOpd = for (g <- grouped) yield {
        g match {
          case List(a, b) => op(a, b)
          case List(x) => x
        }
      }
      nonassocPairwiseReduce(pairwiseOpd, op)
    }
  }
}


class AdditiveRT(numAddends: Int, bitWidth: Int) extends Module {

    require(numAddends >= 1, "Number of addends must be at least one.")
    require(bitWidth >= 1, "Bitwidth must be at least one.")
    
    val io = IO(new Bundle {
        val in  = Input (Vec(numAddends, SInt(bitWidth.W)))
        val out = Output(SInt(bitWidth.W))
    })
    
    io.out := nonassocPairwiseReduce(io.in toList, (x: SInt, y: SInt) => x + y)
}

defined function nonassocPairwiseReduce
defined class AdditiveRT

### Putting them Together

Together, the parallel multiplier and additive reduction tree perform an n-dim inner product. They are simply connected output to input. During use of the PE, the bypass type of the IPU is dictated by the functionality of the ALU.

In [7]:
def checkparamsIPU(width: Int, bypass: String, bitwidth: Int) {
    require(width >= 1, "Width must be at least one.")
    require(List("None", "Firm").contains(bypass), "Bypass must be \"None\" or \"Firm\"")
    require(bitwidth >= 0, "Data bitwidth must be non-negative")
}


class IPU(width: Int, bypass: String, bitwidth: Int) extends Module {
    
    checkparamsIPU(width, bypass, bitwidth)
    
    val io = IO(new Bundle {
        val in1 = Input(Vec(width, SInt(bitwidth.W)))
        val in2 = Input(Vec(width, SInt(bitwidth.W)))
        val out = Output(UInt(bitwidth.W))
        val sel = if(bypass == "Firm") Some(Input(Vec(width, Bool()))) else None
        val bp1 = if(bypass == "Firm") Some(Output(UInt(bitwidth.W)))  else None
        val bp2 = if(bypass == "Firm") Some(Output(UInt(bitwidth.W)))  else None
    })
    
    val pM = new pMultiplier(width, bitwidth)
    pM.io.in1 := io.in1
    pM.io.in2 := io.in2
    
    val aRT = new AdditiveRT(width, bitwidth)
    aRT.io.in := pM.io.out
    
    io.out := aRT.io.out
    
    if (bypass == "Firm") {
        io.bp1.get := PriorityMux(io.sel.get, io.in1)
        io.bp2.get := PriorityMux(io.sel.get, io.in2)
    }
}

defined function checkparamsIPU
defined class IPU

## ALU

The ALU is configurable based on the functions it should support and the data bitwidth. The minimum ALU simply connects the input and the output directly. Accumulate means add the local register file output to the inner product from the IPU. Add and Max do perform their respective operation on the two bypasses from the IPU.

Selecting the operation to perfom is done with a one-hot select signal. The order is always Identity, Add, Max, then Accumulate, but the exact indices will change depending on which are supported.

There are plans to add bypass behavior. Right now, it is supported with an identity operation.

In [8]:
def checkparamsALU(funcs: List[String], datawidth: Int) {
    require(funcs.contains("Identity"), "ALU functions must explicitly include Identity.")
    val supportedFuncs = List("Identity", "Add", "Max", "Accumulate")
    for(x <- funcs)(require(supportedFuncs.contains(x), "Unsupported Function"))
}

class ALU(funcs: List[String], datawidth: Int) extends Module {
    
    checkparamsALU(funcs, datawidth)
 
    val io = IO(new Bundle {
        val innr_prod = Input(SInt(datawidth.W))
        val func_slct = Input(Vec(funcs.length, Bool()))
        val output    = Output(SInt(datawidth.W))
        val weight_bp = if(List("Add", "Max").contains(funcs)) Some(Input(SInt(datawidth.W))) else None
        val actvtn_bp = if(List("Add", "Max").contains(funcs)) Some(Input(SInt(datawidth.W))) else None
        val rf_feedbk = if(funcs.contains("Accumulate"))       Some(Input(SInt(datawidth.W))) else None
    })
    
    val idnOut = Some(Wire(SInt(datawidth.W)))
    val addOut = if(funcs.contains("Add"))        Some(Wire(SInt(datawidth.W))) else None
    val maxOut = if(funcs.contains("Max"))        Some(Wire(SInt(datawidth.W))) else None
    val accOut = if(funcs.contains("Accumulate")) Some(Wire(SInt(datawidth.W))) else None
    
    idnOut.get := io.innr_prod
    
    if (funcs.contains("Add")       ) { addOut.get := io.weight_bp.get + io.actvtn_bp.get }
    if (funcs.contains("Accumulate")) { accOut.get := io.innr_prod + io.rf_feedbk.get }
    if (funcs.contains("Max")       ) {
        when (io.weight_bp.get > io.weight_bp.get) {
            maxOut.get := io.weight_bp.get
        } .otherwise {
            maxOut.get := io.actvtn_bp.get
        }
    }
    
    val inters = (idnOut:: addOut :: maxOut :: accOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.output := PriorityMux(io.func_slct, inters)
}

defined function checkparamsALU
defined class ALU

## Feedback Register File

The feedback register file is just a pRF of size 1 as implemented earlier. Its bypass behavior is key to implementing architectures such as FlexFlow. A Hard bypass will connect the input from the NoC to the NLU/ALU and the input from the ALU to the NoC.

## Nonlinear Unit

The Nonlinear Unit (NLU) performs the nonlinear activations functions present in neural networks. As of right now, it supports the Identity function and ReLu, but there are plans to implement sigmoid and tanh using LUTs and linear interpolation.

In [9]:
def checkparamsNLU(funcs: List[String], datawidth: Int) {
    require(funcs.contains("Identity"), "NLU functions must explicitly include Identity.")
    val supportedFuncs = List("Identity", "ReLu")
    for(x <- funcs)(require(supportedFuncs.contains(x), "Unsupported Function"))
}

class NonlinearUnit(funcs: List[String], datawidth: Int) extends Module {
    
    checkparamsNLU(funcs, datawidth)
    
    val io = IO(new Bundle {
        val input = Input(SInt(datawidth.W))
        val fslct = Input(Vec(funcs.length, Bool()))
        val outpt = Output(SInt(datawidth.W))
    })
    
    val idntOut = Some(Wire(SInt(datawidth.W)))
    val reluOut = if(funcs.contains("ReLu")) Some(Wire(SInt(datawidth.W))) else None
    
    idntOut.get := io.input
    if (funcs.contains("ReLu")) {
        when (io.input > 0.S) {
            reluOut.get := io.input
        } .otherwise {
            reluOut.get := 0.S
        }
    }
    
    val inters = (idntOut :: reluOut :: Nil) filter ( _.isDefined ) map ( _.get )
    io.outpt := PriorityMux(io.fslct, inters)
}

defined function checkparamsNLU
defined class NonlinearUnit

## Control

Control is implemented via a State Machine (Moore) and Decoder. The state machine must have its states and state transitions configured. The Decoder must have its input-output map configured.

### State Machine

The state machine takes a map as a parameter and constructs the appropriate hardware implementation. The State Machine is configured via the "nextState" parameter: a map from the current state and current input to the next state.

In [10]:
class StateMachine(nextState: Map[(UInt, UInt), UInt], ctrlWidth: Int) extends Module {
    
    val stateWidth: Int = log2Up(nextState.size)
    
    val io = IO(new Bundle {
        val control = Input (UInt(ctrlWidth.W ))
        val state   = Output(UInt(stateWidth.W))
    })
    
    val register = RegInit(0.U(stateWidth.W))
    register := nextState((io.state, io.control))
    io.state := register
}

defined class StateMachine

### Decoder

The decoder acts as is typical in processors: it converts the state information into all the control signals associated with the appropriate state. It must be configured via the "decode" parameter, a map from the state and control signal name to the desired value.

In [12]:
class Decoder(decode: (UInt, String) => Data,
              statewidth: Int, ports: Int, 
              datawidth: Int, addrwidth: Int,
              aluFuncs: List[String], 
              nluFuncs: List[String]) extends Module {
    
    val io = IO(new Bundle {
        
        val state = Input(UInt(statewidth.W))
        
        val weightRF_wen         = Output(Vec(ports, Bool()))
        val weightRF_ren         = Output(Vec(ports, Bool()))
        val weightRF_waddr       = Output(Vec(ports, UInt(addrwidth.W)))
        val weightRF_raddr_int   = Output(Vec(ports, UInt(addrwidth.W)))
        val weightRF_raddr_ext   = Output(Vec(ports, UInt(addrwidth.W)))
        val weightRF_bp_slct_get = Output(Vec(ports, Bool()))
        
        val actvtnRF_wen         = Output(Vec(ports, Bool()))
        val actvtnRF_ren         = Output(Vec(ports, Bool()))
        val actvtnRF_waddr       = Output(Vec(ports, UInt(addrwidth)))
        val actvtnRF_raddr_int   = Output(Vec(ports, UInt(addrwidth)))
        val actvtnRF_raddr_ext   = Output(Vec(ports, UInt(addrwidth)))
        val actvtnRF_bp_slct_get = Output(Vec(ports, Bool()))
        
        val ipu_sel_get   = Output(Vec(ports, Bool()))
        
        val alu_func_slct = Output(Vec(aluFuncs.length, Bool()))
        
        val intrnlRF_write_en    = Output(Vec(ports, Bool()))
        val intrnlRF_read_en     = Output(Vec(ports, Bool()))
        val intrnlRF_waddr       = Output(Vec(ports, UInt(addrwidth)))
        val intrnlRF_raddr_int   = Output(Vec(ports, UInt(addrwidth)))
        val intrnlRF_raddr_ext   = Output(Vec(ports, UInt(addrwidth)))
        val intrnlRF_bp_slct_get = Output(Vec(ports, Bool()))
        val intrnlRF_wdata_slct  = Output(Bool())
        
        val nlu_func_slct = Output(Vec(nluFuncs.length, Bool()))
    })
    
    io.weightRF_wen         := decode(io.state, "weightRF_wen")
    io.weightRF_ren         := decode(io.state, "weightRF_ren")
    io.weightRF_waddr       := decode(io.state, "weightRF_waddr")
    io.weightRF_raddr_int   := decode(io.state, "weightRF_raddr_int")
    io.weightRF_raddr_ext   := decode(io.state, "weightRF_raddr_ext")
    io.weightRF_bp_slct_get := decode(io.state, "weightRF_bp_slct_get")
}

defined class Decoder

## Putting it all Together

In [18]:
class nPE(stateMap: Map[(UInt, UInt), UInt], extrnl_ctrl_width: Int, // State Machine
          decode: (UInt, String) => Data, RFports: Int, weightRFBP: String, actvtnRFBP: String, datawidth: Int, addrwidth: Int,
          aluFuncs: List[String], nluFuncs: List[String], intrnlRFBP: String
         ) extends Module {
    
    val io = IO(new Bundle {
        val extrnl_ctrl   = Input (SInt(extrnl_ctrl_width.W))
        val weightRF_in   = Input (Vec(RFports, SInt(datawidth.W)))
        val actvtnRF_in   = Input (Vec(RFports, SInt(datawidth.W)))
        val intrnlRF_in   = Input (SInt(datawidth.W))
        val weightRF_2NoC = Output(Vec(RFports, SInt(datawidth.W)))
        val actvtnRF_2NoC = Output(Vec(RFports, SInt(datawidth.W)))
        val intrnlRF_2NoC = Output(SInt(datawidth.W))
        val output        = Output(SInt(datawidth.W))
    })
    
    val stateMachine = new StateMachine(stateMap, extrnl_ctrl_width)
    stateMachine.io.control := io.extrnl_ctrl
    
    val decoder = new Decoder(decode, log2Up(stateMap.size), 
                              RFports, datawidth, addrwidth, aluFuncs, nluFuncs)
    decoder.io.state := stateMachine.io.state
    
    
    // Weight RF
    val weightRF = new pRF(RFports, weightRFBP, datawidth, addrwidth)
    
    // Mandatory Control
    weightRF.io.write_en    := decoder.io.weightRF_wen
    weightRF.io.read_en     := decoder.io.weightRF_ren
    weightRF.io.waddr       := decoder.io.weightRF_waddr
    weightRF.io.raddr_int   := decoder.io.weightRF_raddr_int
    weightRF.io.raddr_ext   := decoder.io.weightRF_raddr_ext
    
    // Optional Control
    if ( weightRF.io.bp_slct.isDefined ) { weightRF.io.bp_slct.get := decoder.io.weightRF_bp_slct_get }
    
    // Mandatory Outputs
    weightRF.io.wdata := io.weightRF_in
    
    // Optional Outputs
    io.weightRF_2NoC  := weightRF.io.rdata_ext
    
    // Activation RF
    val actvtnRF = new pRF(RFports, actvtnRFBP, datawidth, addrwidth)
    
    // Mandatory Control
    actvtnRF.io.write_en    := decoder.io.actvtnRF_wen
    actvtnRF.io.read_en     := decoder.io.actvtnRF_ren
    actvtnRF.io.waddr       := decoder.io.actvtnRF_waddr
    actvtnRF.io.raddr_int   := decoder.io.actvtnRF_raddr_int
    actvtnRF.io.raddr_ext   := decoder.io.actvtnRF_raddr_ext
    
    // Optional Control
    if ( actvtnRF.io.bp_slct.isDefined ) { actvtnRF.io.bp_slct.get := decoder.io.actvtnRF_bp_slct_get }
    
    // Mandatory Outputs
    actvtnRF.io.wdata := io.weightRF_in
    
    // Optional Outputs
    io.actvtnRF_2NoC     := actvtnRF.io.rdata_ext
       
    val ipuBP = if(aluFuncs.contains("Add") || aluFuncs.contains("Max")) "Firm" else "None" 
    val ipu   = new IPU(RFports, ipuBP, datawidth)
    if (ipu.io.sel.isDefined) { ipu.io.sel.get := decoder.io.ipu_sel_get }
    ipu.io.in1 := weightRF.io.rdata_int
    ipu.io.in2 := actvtnRF.io.rdata_int
    
    val alu = new ALU(aluFuncs, datawidth)
    alu.io.func_slct := decoder.io.alu_func_slct
    alu.io.innr_prod := ipu.io.out
    if(alu.io.weight_bp.isDefined) alu.io.weight_bp.get := ipu.io.bp1.get
    if(alu.io.actvtn_bp.isDefined) alu.io.actvtn_bp.get := ipu.io.bp2.get
    
    val intrnlRF = new pRF(1, intrnlRFBP, datawidth, addrwidth)
    intrnlRF.io.write_en  := decoder.io.intrnlRF_write_en
    intrnlRF.io.read_en   := decoder.io.intrnlRF_read_en
    intrnlRF.io.waddr     := decoder.io.intrnlRF_waddr
    intrnlRF.io.raddr_int := decoder.io.intrnlRF_raddr_int
    intrnlRF.io.raddr_ext := decoder.io.intrnlRF_raddr_ext
    if (intrnlRF.io.bp_slct.isDefined) { intrnlRF.io.bp_slct.get := decoder.io.intrnlRF_bp_slct_get }
    intrnlRF.io.wdata := Mux(decoder.io.intrnlRF_wdata_slct, alu.io.output, io.intrnlRF_in)
    io.intrnlRF_2NoC := intrnlRF.io.rdata_ext
    if(alu.io.rf_feedbk.isDefined) alu.io.rf_feedbk.get := intrnlRF.io.rdata_int
    
    val nlu = new NonlinearUnit(nluFuncs, datawidth)
    nlu.io.fslct := decoder.io.nlu_func_slct
    nlu.io.input     := intrnlRF.io.rdata_int
    io.output        := nlu.io.outpt
    
    // Woot woot
}

defined class nPE

## Future Plans

Basing the nPE off bypasses (i.e. subtractive modifications to a preconfigured topology) is limiting. A constructive approach would be much more general. This should be accomplished through a DSL and much more fully utilizing the functional programming capabilities of Scala.